In [11]:
import os
import xarray as xr
import pandas as pd

#Files
data_dir = "/Net/elnino/data/obs/ERA5/global/daily/"
# Example: '/Net/elnino/data/obs/ERA5/global/daily/mslp_era5_day_2002.nc'

# List all files 
all_files = os.listdir(data_dir)

#file_info_list = []
file_info_dict = {}
#file_info = pd.DataFrame(columns=["filename", "year", "variable"])



for filename in all_files:
    #
    parts = filename.split("_")
    year = int(parts[-1].split(".")[0])
    variable = parts[0]
    

    #file_info_list.append((filename, year, variable))
    # If the variable is not already a key in the dictionary, initialize an empty DataFrame
    if variable not in file_info_dict:
        file_info_dict[variable] = pd.DataFrame(columns=["filename", "year"])
    
    # Append information to the DataFrame associated with the variable
    #file_info_dict[variable] = file_info_dict[variable].append({"filename": filename, "year": year}, ignore_index=True)
    new_df = pd.DataFrame({"filename": [filename], "year": [year]})
    file_info_dict[variable] = pd.concat([file_info_dict[variable], new_df], ignore_index=True)

#file_info = pd.DataFrame(file_info_list, columns=["filename", "year", "variable"])
# dictionary to store grouped dataArrays for each year
data_by_year = {}

print("Debugging landmark 1")

# Iterate over each distinct variable in file_info_dict
for variable_name, variable_info in file_info_dict.items():
    # Iterate over unique years for the current variable
    for year in variable_info["year"].unique():
        # Filter files for the current year and the previous year
        year_files = variable_info[(variable_info["year"] == year) | (variable_info["year"] == year - 1)]["filename"].tolist()
    
        # Initialize an empty list to store DataArrays for the current variable
        variable_data = []
        print("Debugging landmark 1.5")
  
        for filename in year_files:
            ds = xr.open_dataset(os.path.join(data_dir, filename))
            variable_data.append(ds[variable_name])
        
        # Concatenate the DataArrays along a new dimension (e.g., variable)
        stacked_data = xr.concat(variable_data, dim="variable")
    
        # Store the stacked DataArray in the dictionary with the year as the key
        if year in data_by_year:
            data_by_year[year][variable_name] = stacked_data
        else:
            data_by_year[year] = {variable_name: stacked_data}

print("Debugging landmark 2")

csv_path = "/unity/f2/aoleksy/MonsoonForecast/onset_pen_FL.csv"
ground_truth = pd.read_csv(csv_path)

for year, gt_value in zip(ground_truth["year"], ground_truth["varToPredict"]):
    if year in data_by_year:
        data_by_year[year]["ground_truth"] = gt_value

print("Debugging landmark 3")

Debugging landmark 1
Debugging landmark 1.5
Debugging landmark 1.5
Debugging landmark 1.5
Debugging landmark 1.5
Debugging landmark 1.5


KeyboardInterrupt: 

In [ ]:
data_by_year = {}

print("Debugging landmark 1")

for year in file_info_dict["year"].unique():
   #current year and previous year so that we can predict onset date further back
    year_files = file_info_dict[(file_info_dict["year"] == year) | (file_info_dict["year"] == year - 1)]["filename"].tolist()
    

    variable_data = []
    print("Debugging landmark 1.5")
  
    for filename in year_files:
   
        ds = xr.open_dataset(os.path.join(data_dir, filename))
        
       
        variable_data.append(ds[variable])
        
    # ***WILL PROBABLY HAVE TO MODIFY THIS***
    stacked_data = xr.concat(variable_data, dim="variable")
    
    
    data_by_year[year] = stacked_data